In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# partly from https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
RUG = pd.read_pickle("../obfuscated_data.pkl")
RUG.interpolate(method='linear', inplace=True)#, limit=20)
RUG = RUG[::10]
dfs = [RUG.filter([i]) for i in RUG]

In [13]:
dfs = [i.dropna() for i in dfs]

In [44]:
# RUG

# Timeseries LSTM

In [16]:
def func(df, name):

    df2 = df.copy()

    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(df2)

    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=3):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)
    
    # reshape into X=t and Y=t+1
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.fit(trainX, trainY, epochs=25, verbose=2, callbacks=[callback], batch_size=64)

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    rmse_train = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    # print('Train Score: %.2f RMSE' % (trainScore))
    rmse_test = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    # print('Test Score: %.2f RMSE' % (testScore))

    mae_train = tf.keras.metrics.mean_absolute_error(trainY[0], trainPredict[:,0]).numpy()
    mae_test = tf.keras.metrics.mean_absolute_error(testY[0], testPredict[:,0]).numpy()

    mape_train = tf.keras.metrics.mean_absolute_percentage_error(trainY[0], trainPredict[:,0]).numpy()
    mape_test = tf.keras.metrics.mean_absolute_percentage_error(testY[0], testPredict[:,0]).numpy()

    return (name, (rmse_train, rmse_test, mae_train, mae_test, mape_train, mape_test))

In [22]:
results = []

for name, df in zip([i for i in RUG], dfs):
    print(name)
    
    if df.isnull().values.any():
        print("NaN")
        results.append((name, (np.nan, np.nan)))
        print("-------------")
        continue

    r = func(df, name)
    print("-------------")
    results.append(r)

Location 1 - flow
Epoch 1/25
3211/3211 - 26s - loss: 0.0068 - mse: 0.0068 - 26s/epoch - 8ms/step
Epoch 2/25
3211/3211 - 16s - loss: 0.0031 - mse: 0.0031 - 16s/epoch - 5ms/step
Epoch 3/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 4/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 5/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 6/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 7/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 8/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 9/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 10/25
3211/3211 - 13s - loss: 0.0031 - mse: 0.0031 - 13s/epoch - 4ms/step
Epoch 11/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 12/25
3211/3211 - 14s - loss: 0.0031 - mse: 0.0031 - 14s/epoch - 4ms/step
Epoch 13/25
3211/3211 - 14s - l

In [43]:
# for i in results:
#     print(i)

In [38]:
lstm_results = [[i[0], i[1][0], i[1][1], i[1][2].numpy(), i[1][3].numpy(), i[1][4].numpy(), i[1][5].numpy()] for i in results]

lstm_results = pd.DataFrame(lstm_results)

lstm_results.columns = ["Location", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", "Train MAPE", "Test MAPE"]
lstm_results.set_index("Location", inplace=True)
lstm_results.sort_index(inplace=True)
lstm_results = lstm_results.astype(float).round(3)

display(lstm_results)

# print(lstm_results.round(3).to_latex())

,Train RMSE,Test RMSE,Train MAE,Test MAE,Train MAPE,Test MAPE
Location,,,,,,
Location 1 - flow,26.040,26.233,14.023,16.383,6.888626e+07,5.753270e+02
Location 10 - flow,17.744,21.021,5.995,6.963,5.162889e+03,2.352626e+03
Location 11 - flow,5.364,2.030,1.050,0.355,2.780048e+08,2.711683e+08
Location 11 - head,26.026,0.584,0.648,0.430,9.780000e-01,8.000000e-01
Location 12 - head,0.686,0.578,0.294,0.273,4.460000e-01,4.130000e-01
Location 2 - consumption,0.323,0.437,0.196,0.304,5.260000e+01,1.718280e+05
Location 3 - consumption,0.405,0.469,0.128,0.199,7.055900e+06,6.871160e+06
Location 4 - consumption,0.356,0.686,0.095,0.306,2.697000e+00,2.163765e+04
Location 5 - consumption,0.207,0.405,0.076,0.279,5.309642e+04,1.864684e+06


In [42]:
cols = pd.MultiIndex.from_product([['Train', 'Test'], ['RMSE', 'MAE', 'MAPE']])
lstm_results.columns = cols
print(lstm_results.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & \multicolumn{3}{r}{Train} & \multicolumn{3}{r}{Test} \\
 & RMSE & MAE & MAPE & RMSE & MAE & MAPE \\
Location &  &  &  &  &  &  \\
\midrule
Location 1 - flow & 26.040000 & 26.233000 & 14.023000 & 16.383000 & 68886264.000000 & 575.327000 \\
Location 10 - flow & 17.744000 & 21.021000 & 5.995000 & 6.963000 & 5162.889000 & 2352.626000 \\
Location 11 - flow & 5.364000 & 2.030000 & 1.050000 & 0.355000 & 278004832.000000 & 271168288.000000 \\
Location 11 - head & 26.026000 & 0.584000 & 0.648000 & 0.430000 & 0.978000 & 0.800000 \\
Location 12 - head & 0.686000 & 0.578000 & 0.294000 & 0.273000 & 0.446000 & 0.413000 \\
Location 2 - consumption & 0.323000 & 0.437000 & 0.196000 & 0.304000 & 52.600000 & 171828.047000 \\
Location 3 - consumption & 0.405000 & 0.469000 & 0.128000 & 0.199000 & 7055900.000000 & 6871160.000000 \\
Location 4 - consumption & 0.356000 & 0.686000 & 0.095000 & 0.306000 & 2.697000 & 21637.650000 \\
Location 5 - consumption & 0.207000 & 0.405